# Quickstart

In this guide, we will train a simple Convolutional Neural Network (CNN) on MNIST handwritten digits dataset. We will simulate a Federated Learning experiment between two collaborators, orchestrated by an aggregator, via few CLI commands.

OpenFL must be installed for this tutorial. Refer to the [installation](https://openfl.readthedocs.io/en/latest/installation.html) guide.

### Create a workspace

We will reuse an existing workspace. OpenFL provides many reference model templates. Check them out under [workspaces](https://github.com/securefederatedai/openfl/tree/develop/openfl-workspace).

In [ ]:
!fx workspace create --prefix ./mnist_example --template keras/mnist
%cd ./mnist_example

### Initialize a Plan

This step builds an entire FL experiment plan, along with the initial set of parameters that will be used in the experiment.
We supply `localhost` as the aggregator address, for simulation purposes. The command below supports models of arbitrary input shapes and initializations (e.g. when using pre-trained models).

For a complete list of supported options, refer to [`fx plan initialize`](https://openfl.readthedocs.io/en/latest/reference/fx.plan.html#fx-plan-initialize).

In [ ]:
!fx plan initialize -a localhost

### Create a certificate authority (CA)

OpenFL supports mTLS, which ensures secure communication between the collaborators and the aggregator. This step generates a certificate authority (CA) that will be used to sign the certificates of the collaborators. The CA is generated only once and can be reused for multiple experiments.

No additional arguments are required for this command.

In [ ]:
!fx workspace certify

### Aggregator Setup

The [`fx aggregator`](https://openfl.readthedocs.io/en/latest/reference/fx.aggregator.html) subcommand options to manage the aggregator-side certificate signing, generation and verification.

Here, we generate the aggregator's certificate and private key. The aggregator's certificate is then signed by the CA generated in the previous step.

For all commands that follow, `--silent` is used to skip manual confirmation. In practice, users are recommended to verify the details before proceeding.

In [ ]:
!fx aggregator generate-cert-request --fqdn localhost
!fx aggregator certify --fqdn localhost --silent

### Collaborator Setup

Following aggregator, [`fx collaborator`](https://openfl.readthedocs.io/en/latest/reference/fx.collaborator.html) subcommand manages the collaborator-side certificate signing, generation and verification. In addition, because collaborators bring their own local datasets, we populate dataset information under `plan/data.yaml`.

#### Setup first collaborator `bob`

This step registers `--data_path` against collaborator's name `-n` under `plan/data.yaml`.

In [ ]:
!fx collaborator create --data_path 0 -n bob --silent

The next step generates `bob`s certificate and key pair.

In [ ]:
!fx collaborator generate-cert-request -n bob --silent

Certify `bob` with the CA.

> Note: Here, we assume CA to be the aggregator. Hence the command below runs on the `aggregator` side. In reality, `bob`'s certificate signing request (CSR) package generated in the last step, is sent to the `aggregator`, which then signs the certificate with the CA. The signed certificate then has to be sent back to `bob`.

In [ ]:
!fx collaborator certify --request-pkg col_bob_to_agg_cert_request.zip --silent

#### Setup second collaborator `charlie`

As before, we create a collaborator by registering `--data_path` against collaborator's name `-n` under `plan/data.yaml`.

In [ ]:
!fx collaborator create --data_path 1 -n charlie --silent

Now, generate `charlie`s certificate and key pair.

In [ ]:
!fx collaborator generate-cert-request -n charlie --silent

Certify `charlie` with the CA.

> Note: Again, we assume CA to be the aggregator. Hence the command below is expected to be run on the `aggregator` side. In practice, `charlie`'s certificate signing request (CSR) package generated in the last step, is sent to the `aggregator`, which then signs the certificate with the CA. The signed certificate then has to be sent back to `charlie`.

In [ ]:
!fx collaborator certify --request-pkg col_charlie_to_agg_cert_request.zip --silent

### Import CA signed certificates

This step imports the CA-signed certificates of the collaborator and the aggregator into the workspace. We are back on the individual collaborator nodes for this step.

In [ ]:
!fx collaborator certify --import agg_to_col_bob_signed_cert.zip
!fx collaborator certify --import agg_to_col_charlie_signed_cert.zip

### Start the simulation

This step starts the simulation of the FL experiment. The aggregator will orchestrate the training process between the collaborators.

In practice, this command is executed on respective aggregator/collaborator nodes.

In [ ]:
!fx aggregator start &\
 fx collaborator start -n bob &\
 fx collaborator start -n charlie

### Next steps

The simulation will run for a few epochs. Once the simulation is complete, the model will be saved under the `save` directory. The model can be converted to framework native formats using the [`fx model save`](https://openfl.readthedocs.io/en/latest/reference/fx.model.html#fx-model-save) command.

For detailed information on options supported via CLI, refer to the [CLI reference](https://openfl.readthedocs.io/en/latest/fx.html).

For an in-depth understanding of the TaskRunner architecture, refer to the [TaskRunner API](https://openfl.readthedocs.io/en/latest/about/features_index/taskrunner.html).